# Clustering Crypto

In [32]:
pip install path

In [34]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [5]:
from google.colab import files
uploaded = files.upload()

Saving crypto_data.csv to crypto_data.csv


In [35]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
crypto_df = pd.read_csv(file_path)
crypto_df

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [36]:
 # Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df[crypto_df.IsTrading != False]
crypto_df

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1243,SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [37]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns=['IsTrading'])
crypto_df

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,SERO,Super Zero,Ethash,PoW,NaN,1000000000
1244,UOS,UOS,SHA-256,DPoI,NaN,1000000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [49]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()
crypto_df

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
1238,ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Equihash,PoW,7.296538e+06,21000000


In [39]:
# Create a new DataFrame that holds only the cryptocurrencies names.
names_df = crypto_df.filter(["CoinName"], axis=1)
names_df = names_df.drop_duplicates()
names_df

,CoinName
0,42 Coin
2,404Coin
4,808
5,EliteCoin
7,Bitcoin
...,...
1238,ZEPHYR
1242,Gapcoin
1245,Beldex
1246,Horizen


In [41]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
#crypto_df = crypto_df.drop(columns=["Unnamed: 0"], axis=1)
crypto_df = crypto_df.drop(columns=["CoinName"], axis=1)
crypto_df

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
1238,ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Equihash,PoW,7.296538e+06,21000000


In [42]:
# Use get_dummies() to create variables for text features.
x = pd.get_dummies(crypto_df)
x

,TotalCoinsMined,Unnamed: 0_1337,Unnamed: 0_1CR,Unnamed: 0_404,Unnamed: 0_42,Unnamed: 0_808,Unnamed: 0_8BIT,Unnamed: 0_AAC,Unnamed: 0_ABJ,Unnamed: 0_ABS,Unnamed: 0_ABY,Unnamed: 0_AC3,Unnamed: 0_ACC,Unnamed: 0_ACM,Unnamed: 0_ACOIN,Unnamed: 0_ACP,Unnamed: 0_ADA,Unnamed: 0_ADC,Unnamed: 0_ADK,Unnamed: 0_ADZ,Unnamed: 0_AEON,Unnamed: 0_AERO,Unnamed: 0_ALPS,Unnamed: 0_ALTCOM,Unnamed: 0_ALX,Unnamed: 0_AM,Unnamed: 0_ANTI,Unnamed: 0_APEX,Unnamed: 0_AQUA,Unnamed: 0_ARC,Unnamed: 0_ARCH,Unnamed: 0_ARCO,Unnamed: 0_ARE,Unnamed: 0_ARG,Unnamed: 0_ARGUS,Unnamed: 0_ARK,Unnamed: 0_ASAFE2,Unnamed: 0_ASN,Unnamed: 0_ATMOS,Unnamed: 0_ATX,...,TotalCoinSupply_76500000,TotalCoinSupply_78000000,TotalCoinSupply_78835200,TotalCoinSupply_7905634,TotalCoinSupply_80000000,TotalCoinSupply_800000000,TotalCoinSupply_8080000000,TotalCoinSupply_81962100,TotalCoinSupply_82003200,TotalCoinSupply_82546564,TotalCoinSupply_84000000,TotalCoinSupply_840000000,TotalCoinSupply_84096000,TotalCoinSupply_850000000,TotalCoinSupply_85000000000,TotalCoinSupply_8634140,TotalCoinSupply_86712634466,TotalCoinSupply_87500000,TotalCoinSupply_88000000,TotalCoinSupply_88188888,TotalCoinSupply_888000000,TotalCoinSupply_88888888,TotalCoinSupply_890000000,TotalCoinSupply_9000000,TotalCoinSupply_90000000,TotalCoinSupply_900000000,TotalCoinSupply_9000000000,TotalCoinSupply_90000000000,TotalCoinSupply_90600000000,TotalCoinSupply_91388946,TotalCoinSupply_92000000000,TotalCoinSupply_92233720000000000,TotalCoinSupply_9354000,TotalCoinSupply_9507271,TotalCoinSupply_9736000,TotalCoinSupply_98000000,TotalCoinSupply_98100000000,TotalCoinSupply_990000000000,TotalCoinSupply_999481516,TotalCoinSupply_9999999
0,4.199995e+01,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.055185e+09,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.000000e+00,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2.927942e+10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1.792718e+07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1238,2.000000e+09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1242,1.493105e+07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1245,9.802226e+08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1246,7.296538e+06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
# Standardize the data with StandardScaler().
crpyto_scaled = StandardScaler().fit_transform(x)
crpyto_scaled

array([[-0.10282804, -0.03823596, -0.03823596, ..., -0.03823596,
        -0.03823596, -0.03823596],
       [-0.07661326, -0.03823596, -0.03823596, ..., -0.03823596,
        -0.03823596, -0.03823596],
       [-0.10282804, -0.03823596, -0.03823596, ..., -0.03823596,
        -0.03823596, -0.03823596],
       ...,
       [-0.0784756 , -0.03823596, -0.03823596, ..., -0.03823596,
        -0.03823596, -0.03823596],
       [-0.10264677, -0.03823596, -0.03823596, ..., -0.03823596,
        -0.03823596, -0.03823596],
       [-0.10282485, -0.03823596, -0.03823596, ..., -0.03823596,
        -0.03823596, -0.03823596]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [16]:
# Using PCA to reduce dimension to three principal components.
# Initialize PCA model
pca = PCA(n_components=3)

In [51]:
# Create a DataFrame with the three principal components.
crypto_pca = pca.fit_transform(crpyto_scaled)
df_crypto_pca = pd.DataFrame(data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
df_crypto_pca.head()

,PC 1,PC 2,PC 3
0,-0.129256,-0.062941,-0.340683
2,-0.053611,-0.030825,-0.258797
4,0.009086,0.018352,-0.021464
5,0.810554,-0.036388,0.151234
7,-0.655401,-0.079887,-0.031989


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [52]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))
# Calculate the inertia for the range of K values
for i in k:
   km = KMeans(n_clusters=i, random_state=0)
   km.fit(df_crypto_pca)
   inertia.append(km.inertia_)

In [53]:
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [57]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=5)
model

# Fit the model
model.fit(df_crypto_pca)

# Predict clusters
predictions = model.predict(df_crypto_pca)
print(predictions)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [58]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df =  pd.concat([crypto_df, df_crypto_pca], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = names_df["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(685, 10)


,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
0,42,Scrypt,PoW/PoS,4.199995e+01,42,-0.129256,-0.062941,-0.340683,42 Coin,0
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.053611,-0.030825,-0.258797,404Coin,0
4,808,SHA-256,PoW/PoS,0.000000e+00,0,0.009086,0.018352,-0.021464,808,0
5,1337,X13,PoW/PoS,2.927942e+10,314159265359,0.810554,-0.036388,0.151234,EliteCoin,0
7,BTC,SHA-256,PoW,1.792718e+07,21000000,-0.655401,-0.079887,-0.031989,Bitcoin,0
8,ETH,Ethash,PoW,1.076842e+08,0,-0.442396,-0.032680,0.032534,Ethereum,0
9,LTC,Scrypt,PoW,6.303924e+07,84000000,-0.624305,-0.068121,-0.258984,Litecoin,0
10,DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.075887,-0.050154,0.074869,Dash,0
11,XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.427732,-0.064422,-0.129464,Monero,0
12,ETC,Ethash,PoW,1.133597e+08,210000000,-0.630587,0.002599,-0.059052,Ethereum Classic,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [62]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"]
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [63]:
# Create a table with tradable cryptocurrencies.
clustered_df[["Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply", "CoinName", "Class" ]].hvplot.table()

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,CoinName,Class]

In [64]:
# Print the total number of tradable cryptocurrencies.
print(f"There are {clustered_df.shape[0]} tradable cryptocurrencies.")

There are 685 tradable cryptocurrencies.


In [65]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.

scaler = MinMaxScaler()
data = scaler.fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
data

array([[4.55364914e-16, 5.94230127e-03],
       [5.76795558e-09, 7.00182308e-03],
       [0.00000000e+00, 5.94230122e-03],
       ...,
       [1.51812440e-08, 6.92655266e-03],
       [2.27682457e-10, 5.94962775e-03],
       [1.08420218e-11, 5.94243008e-03]])

In [66]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
last_df = pd.DataFrame(data, columns=["TotalCoinSupply", "TotalCoinsMined"], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
last_df["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
last_df["Class"] = clustered_df["Class"]

last_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
0,4.553649e-16,0.005942,42 Coin,0
2,5.767956e-09,0.007002,404Coin,0
4,0.000000e+00,0.005942,808,0
5,3.406122e-06,0.035342,EliteCoin,0
7,2.276825e-10,0.005960,Bitcoin,0
8,0.000000e+00,0.006050,Ethereum,0
9,9.107298e-10,0.006006,Litecoin,0
10,2.385245e-10,0.005951,Dash,0
11,0.000000e+00,0.005960,Monero,0
12,2.276825e-09,0.006056,Ethereum Classic,0


In [67]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
last_df.hvplot.scatter( 
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class"
)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)